
Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила меня подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Импортируем библиотеки, сделаем запрос в sql lite, сохраним необходимую нам таблицу и выгрузим ее для создания дашборда

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT user_id, lt_day, age,case when gender_segment = 1 then 'f' when gender_segment = 0 then 'm' end gender_segment,
case when lt_day <=365 then 'New' else 'Old' end is_new, os_name, cpe_type_name, country, city, 
substr(ags.title, 4) age_segment, substr(ts.title, 4) traffic_segment, substr(ls.title, 4) lifetime_segment, nps_score, 
case when nps_score between 9 and 10 then 'promoters' when nps_score between 7 and 8 then 'passives' else 'detractors' end nps_group
FROM user u
left join location l on u.location_id= l.location_id
left join age_segment ags on u.age_gr_id = ags.age_gr_id
left join traffic_segment ts on u.tr_gr_id = ts.tr_gr_id
left join lifetime_segment ls on u.lt_gr_id = ls.lt_gr_id
;
"""

In [4]:
df = pd.read_sql(query, engine)
df.sample(3)

,user_id,lt_day,age,gender_segment,is_new,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
209223,KT7TW3,862,26.0,f,Old,ANDROID,SMARTPHONE,Россия,Томск,25-34,25-30,25-36,9,promoters
133563,GVWTLJ,1440,68.0,m,Old,IOS,SMARTPHONE,Россия,Махачкала,66 +,30-35,36+,10,promoters
139464,H6YGL0,2394,42.0,f,Old,ANDROID,SMARTPHONE,Россия,Новосибирск,35-44,0.1-1,36+,5,detractors


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/pavel2662/viz/NPS_16472384343490/Dashboard1?publish=yes

### Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/6Z6lexbSStwetg